# Combining Data for Final Database

In [5]:
import pandas as pd
import json
import os

In [11]:
DATA_FOLDER = "../data/preprocessed_sustainability_reports/"

In [23]:
[file for file in os.listdir(DATA_FOLDER)]

['all_asset_managers_preprocessed_vfinal_BERT.json',
 '.DS_Store',
 'all_pension_funds_preprocessed_vfinal_BERT.json',
 'all_asian_banks_preprocessed_vfinal_BERT.json',
 'all_pension_funds_preprocessed_vfinal.json',
 'all_vfinal_BERT.json',
 'all_asset_managers_preprocessed_vfinal.json',
 'all_insurance_preprocessed_vfinal_BERT.json',
 'all_vfinal.json',
 'all_asian_banks_preprocessed_vfinal.json',
 'all_insurance_preprocessed_vfinal.json']

### Combine all 4 json into 1

In [16]:
files = [file for file in os.listdir(DATA_FOLDER) if "BERT" not in file and ".DS" not in file]
files_BERT = [file for file in os.listdir(DATA_FOLDER) if "BERT" in file]

def merge_JsonFiles(filename):
    result = list()
    for f1 in filename:
        with open(DATA_FOLDER + f1, 'r') as infile:
            result.extend(json.load(infile))

    with open(DATA_FOLDER +'all_vfinal.json', 'w') as output_file:
        json.dump(result, output_file)

merge_JsonFiles(files)

In [26]:
def merge_JsonFiles(filename):
    result = list()
    for f1 in filename:
        with open(DATA_FOLDER + f1, 'r') as infile:
            result.extend(json.load(infile))

    with open(DATA_FOLDER + 'all_vfinal_BERT.json', 'w') as output_file:
        json.dump(result, output_file)

merge_JsonFiles(files_BERT)

In [25]:
with open(DATA_FOLDER + 'all_vfinal_BERT.json', 'r') as infile:
    all_data = json.load(infile)

In [63]:
len(all_data)

236

### Combine text data predictions with charts

In [120]:
DATA_FOLDER = "../data/dashboard_data/"
with open(DATA_FOLDER + 'all_text_output.json','r') as infile:  
    text_predictions = json.load(infile)

with open(DATA_FOLDER + 'all_chart_output.json','r') as infile:  
    chart_predictions = json.load(infile)
        

In [121]:
text_charts = []
count = 0
count_a = 0
for company in text_predictions:
    company_t = company["company"]
    year_t = company["year"]
    flag = 0
    for company_charts in chart_predictions:
        company_c = company_charts["company"]
        year_c = company_charts["year"]
        
        if company_t == company_c and year_t == year_c:
            charts_c = company_charts["images_path"]
            company["charts_images"] = charts_c
            text_charts.append(company)
            flag = 1
            count+=1
            break
           
    if flag == 0 :
        count_a +=1
        print(company)
        company["charts_images"] = {}
        text_charts.append(company)

{'company': 'Citic Capital', 'year': '2019', 'url': 'http://www.citiccapital.com/News%20Files/CCHL%20ESG%20Report%202019%20(EN).pdf', 'text_output': {'page': ['7'], 'sentence': ['In our Hong Kong office, we fill the space with plants, which we believe can improve the air quality.'], 'relevance_prob': [0.518886483]}}
{'company': 'Capital International', 'year': '2020', 'url': 'https://www.capitalgroup.com/content/dam/cgc/tenants/eacg/esg/files/esg-brochure_2020(en).pdf', 'text_output': {'page': [], 'sentence': [], 'relevance_prob': []}}
{'company': 'Edward Jones Investments', 'year': '2021', 'url': 'https://www.edwardjones.com/sites/default/files/acquiadam/2021-09/CAU-12493A-A_2021-PIC-report.pdf', 'text_output': {'page': ['23'], 'sentence': ['From our work to develop carbon-footprint metrics to green building and sustainable best practices at our home office campuses, Edward Jones is working to safeguard our associates, reduce waste and protect and restore natural habitats.'], 'relevan

In [122]:
len(text_charts)

236

In [123]:
with open(DATA_FOLDER +'all_text_chart_output.json', 'w') as output_file:
    json.dump(text_charts, output_file)


### Combine text & chart data with tables

In [114]:
with open(DATA_FOLDER + 'asian_banks_output_vfinal.json','r') as infile:  
    table_predictions = json.load(infile)

In [116]:
table_predictions[0].keys()

dict_keys(['company', 'year', 'pdf_url', 'table_keywords', 'table_images'])

In [117]:
text_charts_tables = []
count = 0
count_a = 0
for company in text_charts:
    company_t = company["company"]
    year_t = company["year"]
    flag = 0
    for company_charts in table_predictions:
        company_c = company_charts["company"]
        year_c = company_charts["year"]
        
        if company_t == company_c and year_t == year_c:
            company["table_keywords"] = company_charts["table_keywords"]
            company["table_images"] = company_charts["table_images"]            
            text_charts.append(company)
            flag = 1
            count+=1
            break
           
#     if flag == 0 :
#         count_a +=1
#         print(company)
#         company["charts_images"] = {}
#         text_charts.append(company)

KeyboardInterrupt: 